# Разные подходы к определению языка

В этой работе, я опробую дла способа обработки (подготовки) текста для написания алгоритма по распознованию языка.
Сперва мы мы опробуем метод токенезации текста, а затем построим n-gramm'ы.

Импортируем необходимы нем модули.

In [38]:
import re
import collections
import wikipedia as wiki
import warnings
from sklearn.metrics import classification_report, confusion_matrix
from itertools import islice, tee

warnings.filterwarnings("ignore")

Выберем 14 языков.

P.S. 

Сперва их было 15, но с греческим вознили проблемы при скачивании текстов.

In [20]:
langs = ['ru', 'be', 'sl', 'mn', 'pl',
       'ja', 'it', 'cs', 'hr',
       'fr', 'uk', 'tr', 'tt', 'sk']

Напишем функцию, которая будет скачивать статьи из википедии для указанного языка.

In [3]:
def get_texts_for_lang(lang, n=10):
    wiki.set_lang(lang)
    wiki_content = []
    pages = wiki.random(n)
    for page_name in pages:
        try:
            page = wiki.page(page_name)
        except:
            continue

        wiki_content.append('{}\n{}'.format(page.title, page.content.replace('==', '')))

    return wiki_content

In [6]:
wiki_texts = {}

Скачиваем тексты и помещаем их в созданный выше словарь

In [13]:
for lang in langs2:
    wiki_texts[lang] = get_texts_for_lang(lang, 100)
    print(lang, len(wiki_texts[lang]))

cs 94
hr 99
fr 93
uk 93
tr 97
tt 96
sk 93


# Сперва попробуем метод токенизации

Напишем свою фенкцию токенизации текста

In [14]:
def tokenize(text):
    text = re.sub(r'[^\w\s]','',text).replace('\n', '')
    text = re.sub(r'\d','',text)
    text = re.sub(r'[\s]{2,}',' ',text)
    return text.split(' ')

Теперь необходимо составить частотный словарь для каждлого языка, а затем проверим получившийся сллварь для какого-нибудь языка. Для примера – Японский.

In [22]:
result_dict = collections.defaultdict(lambda: collections.defaultdict(lambda: 0))

for lang in langs:
    for article in wiki_texts[lang]:
        for word in tokenize(article.replace('\n', '').lower()):
            result_dict[lang][word] += 1

for lang in wiki_texts.keys():
    print('{}\t{}\t{}'.format(lang, sum(result_dict[lang].values()), len(result_dict[lang])))

            
for lang in wiki_texts.keys():
    summ = sum(result_dict[lang].values())
    for word in result_dict[lang]:
        result_dict[lang][word] = result_dict[lang][word]/summ

for lang in wiki_texts.keys():
    result_dict[lang] = {i[0]:i[1] for i in collections.Counter(result_dict[lang]).most_common(500)}
    

for word in sorted(result_dict['ja'], key=lambda w: result_dict['ja'][w], reverse=True)[:5]:
    print('{}\t{}'.format(result_dict['ja'][word], word))

tr	16539	7266
ru	40710	15245
sl	18284	7844
uk	24559	10804
it	46408	11444
sk	12275	5489
tt	9391	3466
pl	19350	8578
fr	41554	10251
hr	21016	8580
be	15875	7181
mn	20034	7973
ja	5128	3906
cs	35301	13381
0.009945397815912636	外部リンク
0.0093603744149766	脚注
0.008775351014040561	isbn
0.0074102964118564745	the
0.007215288611544462	年月日


Отлично, дальше напишем функцию которая будет предсказывать язык, по получившейся дате. И сразу же попробуем определить язык, для какого-нибудь простого языка.

In [23]:
def make_dict_prediction(text, data):
    words = {}
    for lang in data.keys():
        count = 0
        for word in tokenize(text):
            if word in data[lang]:
                count += data[lang][word]
        words[lang] = count
    
    result = max(words, key=lambda x: words[x]) 
    
    return result

In [24]:
make_dict_prediction('io non parlo italiano', result_dict)

'it'

Неплохо, теперь составим репорт по классификации, с помощью эстиматора, который мы написали.

In [30]:
target = []
prediction = []

for lang in wiki_texts.keys():
    for text in wiki_texts[lang]:
        target.append(lang)
        prediction.append(make_dict_prediction(text, result_dict))

print(classification_report(target, prediction))
print(confusion_matrix(target, prediction))

             precision    recall  f1-score   support

         be       0.97      0.99      0.98        96
         cs       0.92      0.72      0.81        94
         fr       0.85      1.00      0.92        93
         hr       0.89      0.97      0.93        99
         it       0.99      0.99      0.99       100
         ja       0.90      0.88      0.89        92
         mn       1.00      0.97      0.98       100
         pl       1.00      0.98      0.99        92
         ru       0.80      0.98      0.88        91
         sk       0.82      0.81      0.81        93
         sl       0.83      0.96      0.89        94
         tr       0.93      0.87      0.90        97
         tt       1.00      0.95      0.97        96
         uk       0.99      0.74      0.85        93

avg / total       0.92      0.92      0.91      1330

[[95  0  0  0  0  1  0  0  0  0  0  0  0  0]
 [ 0 68  1  3  0  0  0  0  0 13  7  2  0  0]
 [ 0  0 93  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  1 96 

# Теперь попробуем метод n-граммов

И сразу же напишем функцию для создания такого n-gramma. (N=3 в нашем случае)

In [33]:
def convert_to_ngramm(text):
    text = text.lower()
    text = re.sub(r'[^\w\s]','',text).replace('\n', '')
    text = re.sub(r'\d','',text)
    text = re.sub(r'[\s]{2,}',' ',text)
    N = 3 
    output = zip(*(islice(seq, index, None) for index, seq in enumerate(tee(text, N))))
    output = [''.join(x) for x in output]
    return output

Теперь создаем словарь с использованием написанной нами функцией.

In [35]:
result_ngramm = collections.defaultdict(lambda: collections.defaultdict(lambda: 0))

for lang in langs:
    for article in wiki_texts[lang]:
        for word in convert_to_ngramm(article.replace('\n', '').lower()):
            result_ngramm[lang][word] += 1

for lang in wiki_texts.keys():
    print('{}\t{}\t{}'.format(lang, sum(result_ngramm[lang].values()), len(result_ngramm[lang])))

            
for lang in wiki_texts.keys():
    summ = sum(result_ngramm[lang].values())
    for word in result_ngramm[lang]:
        result_ngramm[lang][word] = result_ngramm[lang][word]/summ

for lang in wiki_texts.keys():
    result_ngramm[lang] = {i[0]:i[1] for i in collections.Counter(result_ngramm[lang]).most_common(500)}
    

for word in sorted(result_ngramm['ja'], key=lambda w: result_ngramm['ja'][w], reverse=True)[:5]:
    print('{}\t{}'.format(result_ngramm['ja'][word], word))

tr	126837	6941
ru	296910	12558
sl	126135	6588
uk	179046	11325
it	304163	6273
sk	85769	7666
tt	69994	7791
pl	142722	8185
fr	261079	7224
hr	143824	6530
be	115961	9518
mn	143779	10609
ja	218955	112782
cs	237512	11115
0.0030645566440592816	ている
0.0028316320705167727	年月日
0.002160261240894248	ていた
0.001991276746363408	してい
0.0018588294398392364	として


Пишем новую функцию для определения языка, уже по новым данным

In [36]:
def make_ngramm_prediction(text, data):
    words = {}
    for lang in data.keys():
        count = 0
        for word in tokenize(text):
            if word in data[lang]:
                count += data[lang][word]
        words[lang] = count
    
    result = max(words, key=lambda x: words[x]) 
    
    return result

И наконец проверяем, какой же результат мы получаем...

In [37]:
target = []
prediction = []

for lang in wiki_texts.keys():
    for text in wiki_texts[lang]:
        target.append(lang)
        prediction.append(make_ngramm_prediction(text, result_ngramm))

print(classification_report(target, prediction))
print(confusion_matrix(target, prediction))

             precision    recall  f1-score   support

         be       0.67      0.59      0.63        96
         cs       0.91      0.82      0.86        94
         fr       0.79      1.00      0.88        93
         hr       0.83      0.56      0.67        99
         it       0.71      0.91      0.80       100
         ja       1.00      0.16      0.28        92
         mn       0.85      0.73      0.78       100
         pl       0.85      0.65      0.74        92
         ru       0.74      0.68      0.71        91
         sk       0.90      0.47      0.62        93
         sl       0.75      0.56      0.64        94
         tr       0.27      0.96      0.42        97
         tt       0.68      0.42      0.52        96
         uk       0.78      0.56      0.65        93

avg / total       0.76      0.65      0.66      1330

[[57  0  1  0  2  0  0  0  7  0  0 17  6  6]
 [ 0 77  2  2  1  0  0  1  0  1  1  9  0  0]
 [ 0  0 93  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  4 55 

_____

# Подеведем итоги...

Для нашей выборки текстов, лучше себя показал алгоритм с использованием токенизации.
Скорее всего это связано с тем, что во многих текстах, которые мы скачали для набора языков находится много вхождений английских слов (английского написания), это можно често увидеть на примере языка 'ja' – японский и 'tr' – турецкий.

Скорее всего, при дополнительной обработке текстов или другим наборе языков, точность метода N-gramm изменилась бы.